In [9]:
import os
import pandas as pd
from decimal import Decimal
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import yfinance as yf


import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns


import requests
import csv
import json

import warnings

In [10]:
load_dotenv()

API_KEY = os.getenv("alpha_vantage_api_key")
NASDAQ_DATA_LINK_API_KEY = os.getenv("NASDAQ_DATA_LINK_API_KEY")

In [11]:
# Addtional setting session
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

In [12]:
# Parameters section

alpha_vantage_api_key = API_KEY # FREE TIER API rate limit is 25 requests per day
alpha_vantage_function = {
    'core':[
        'TIME_SERIES_INTRADA'
        ,'TIME_SERIES_DAILY' # this is daily time series quote
        ,'TIME_SERIES_DAILY_ADJUSTED' # this is daily time series adjusted by split/dividend-adjusted
        ,'GLOBAL_QUOTE'
    ]
    ,'fundmental':[
    'INCOME_STATEMENT'
    ,'BALANCE_SHEET' # this is daily time series quote
    ,'CASH_FLOW' # this is daily time series adjusted by split/dividend-adjusted
    ,'EARNINGS'
    ,'EARNINGS_CALENDAR'
]
}

# Define the ticker symbols as a list; eg. TSM,MSFT,AMZN
# ticker_symbols = input("Enter stock tickers separated by commas:") 
# ticker_symbols = ticker_symbols.split(',')
ticker_symbols = [

    # energy
    'SLB'
    ,'HAL'
    # ,'OKE'

    # consumer staples
    # ,'TGT'
    # ,'STZ'
    ,'DLTR'
    # ,'DG'
    # ,'TSN'
    # ,'MNST'
    # ,'PG'
    # ,'PM'
    # ,'MO'


    # finanace
    ,'TRV'

    # technology

    # holding stock
    ,'TSM'
    ,'ACN'
    ,'DHI'
    ,'STZ'
    ,'GOOG'
    ,'TGT'
]
# Time intelligent parameters
window_days = 90
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)
earning_calendar = [
    3  # this will return next 1 qtr forecast earning; nowadays the earning calendar only shows the next 1 qtr forecast earning
    ,6  # this will return next 2 qtr forecast earning
    ,12  # this will return next 4 qtr forecast earning
]

PE_yr_range = 6 # this will return x-1 yr PE range

ticker_dict = {}

# PE TTM Valuation

In [5]:
# Daily quote section
for symbol in ticker_symbols:


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}&outputsize=full'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':


            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': f'{symbol}'
                    }
                ,inplace=True
                )
            
            Daily_stock_df[f'{symbol}'] = Daily_stock_df[f'{symbol}'].astype(str).apply(lambda x: float(x))
            Daily_stock_df[f'{symbol}'] = Daily_stock_df[f'{symbol}'].round(2)


    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)


    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': f'{symbol}'
            }
        ,inplace=True
        )

    Monthly_stock_df[f'{symbol}'] = Monthly_stock_df[f'{symbol}'].astype(str).apply(lambda x: float(x))
    Monthly_stock_df[f'{symbol}'] = Monthly_stock_df[f'{symbol}'].round(2)



    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - 6) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue

            annualEPS_df[f'{symbol}_PE'] = Monthly_stock_df[f'{symbol}'].values / annualEPS_df['reportedEPS'].values
            annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"] = annualEPS_df[f"{symbol}_PE"].mean().round(2)
            annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"] = np.std(annualEPS_df[f"{symbol}_PE"]).round(2)
            annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_+"] = (annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"] + annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围上限
            annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_-"] = (annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"] - annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围下限



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[f'{col}'] = qtrEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue


    # # forecast 1 qtr earnings from alpha vantage API
    # for i in earning_calendar: comment out the for loop in case of future usage, i can be the parameter of {}month
    CSV_URL = f'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={symbol}&horizon=12month&apikey={alpha_vantage_api_key}'
    with requests.Session() as s:
        download = s.get(CSV_URL)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)

        forecast_earanings_df = pd.DataFrame(
            columns=my_list[0]
            ,data=my_list[1::]
            )
        
        if forecast_earanings_df['estimate'].head(1).values != '':
            latest_projected_EPS = float(forecast_earanings_df['estimate'].head(1).values)
        else:
            latest_projected_EPS = 0


    # forecast 1 year earnings from yf API
    yf_data = yf.Ticker(symbol)
        

    EPS_12month_projected = yf_data.info['forwardEps'] # 代表了截止下一个日历年结束的EPS, next year forecasted EPS
    PE_12month_projected = yf_data.info['forwardPE'] # 代表了截止下一个日历年结束的PE, next year forecasted PE
    PEG_12month_projected = yf_data.info['pegRatio'] # 代表了截止下一个日历年结束的PEG 


    # Consolidated section
    df_stock_consolidate = Daily_stock_df.head(window_days)


    df_stock_consolidate_date = df_stock_consolidate.index
    for i in df_stock_consolidate_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate']<= i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        df_stock_consolidate.loc[i, f"{symbol}_EPS_TTM"] = EPS_TTM

        if i == max(df_stock_consolidate.index):
            EPS_latest_projected = latest_projected_EPS + past_3_qtrs_EPS['reportedEPS'].values.sum()  # This metrics is the past 3 qtrs post EPS + 1 projected EPS
        else:
            continue

        df_stock_consolidate[f"{symbol}_EPS_latest_projected"] = EPS_latest_projected
    
    # stock's stats
    df_stock_consolidate[f"{symbol}_PE_TTM"] = (df_stock_consolidate[symbol] / df_stock_consolidate[f"{symbol}_EPS_TTM"]).round(2)
    df_stock_consolidate[f"{symbol}_PE_TTM_avg"] = df_stock_consolidate[f"{symbol}_PE_TTM"].mean().round(2)
    df_stock_consolidate[f"{symbol}_PE_TTM_std"] = np.std(df_stock_consolidate[f"{symbol}_PE_TTM"]).round(2)
    df_stock_consolidate[f"{symbol}_PE_TTM_volatility_+"] = (df_stock_consolidate[f"{symbol}_PE_TTM_avg"] + df_stock_consolidate[f"{symbol}_PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    df_stock_consolidate[f"{symbol}_PE_TTM_volatility_-"] = (df_stock_consolidate[f"{symbol}_PE_TTM_avg"] - df_stock_consolidate[f"{symbol}_PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    df_stock_consolidate[f"{symbol}_relative_valuation_TTM_+"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_+"] * df_stock_consolidate[f"{symbol}_EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    df_stock_consolidate[f"{symbol}_relative_valuation_TTM_-"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_-"] * df_stock_consolidate[f"{symbol}_EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    df_stock_consolidate[f"{symbol}_relative_valuation_TTM_median"] = (np.median([df_stock_consolidate[f"{symbol}_relative_valuation_TTM_+"], df_stock_consolidate[f"{symbol}_relative_valuation_TTM_-"]])).round(2) #这个是根据最新TTM PE估值的价格中位数

    df_stock_consolidate[f"{symbol}_relative_valuation_nextQuater_projected_+"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_+"] * df_stock_consolidate[f"{symbol}_EPS_latest_projected"]).round(2) # 这个是relative valuation的价格上限
    df_stock_consolidate[f"{symbol}_relative_valuation_nextQuater_projected_-"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_-"] * df_stock_consolidate[f"{symbol}_EPS_latest_projected"]).round(2) # 这个是relative valuation的价格下限
    df_stock_consolidate[f"{symbol}_relative_valuation_nextQuater_projected_median"] = (np.median([df_stock_consolidate[f"{symbol}_relative_valuation_nextQuater_projected_+"], df_stock_consolidate[f"{symbol}_relative_valuation_nextQuater_projected_-"]])).round(2) #这个是根据3 qtrs post EPS + 1 projected EPS 得出PE估值的价格中位数

    df_stock_consolidate[f"{symbol}_{window_days}_price_min"] = df_stock_consolidate[symbol].min().round(2)
    df_stock_consolidate[f"{symbol}_{window_days}_price_max"] = df_stock_consolidate[symbol].max().round(2)
    df_stock_consolidate[f"{symbol}_{window_days}_price_avg"] = df_stock_consolidate[symbol].mean().round(2)
    df_stock_consolidate[f"{symbol}_{window_days}_price_std"] = np.std(df_stock_consolidate[symbol]).round(2)

    df_stock_consolidate[f"{symbol}_PE_{PE_yr_range-1}yr_avg"] = annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_avg"].values[0]
    df_stock_consolidate[f"{symbol}_PE_{PE_yr_range-1}yr_std"] = annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_std"].values[0]
    df_stock_consolidate[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_+"] = annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_+"].values[0]
    df_stock_consolidate[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_-"] = annualEPS_df[f"{symbol}_PE_{PE_yr_range-1}yr_volatility_-"].values[0]


    df_stock_consolidate[f"{symbol}_relative_valuation_nextYear_projected_+"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_+"] * EPS_12month_projected).round(2) # 这个是relative valuation的价格上限
    df_stock_consolidate[f"{symbol}_relative_valuation_nextYear_projected_-"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_-"] * EPS_12month_projected).round(2) # 这个是relative valuation的价格下限
    df_stock_consolidate[f"{symbol}_relative_valuation_nextYear_projected_median"] = (np.median([df_stock_consolidate[f"{symbol}_relative_valuation_nextYear_projected_+"], df_stock_consolidate[f"{symbol}_relative_valuation_nextYear_projected_-"]])).round(2) #这个是根据next year projected EPS 得出PE估值的价格中位数

    df_stock_consolidate[f"{symbol}_next12months_PEG"] = PEG_12month_projected
    df_stock_consolidate[f"{symbol}_TTM_PEG"] = (df_stock_consolidate[f"{symbol}_PE_TTM"] / (((EPS_12month_projected - df_stock_consolidate[f"{symbol}_EPS_TTM"]) / df_stock_consolidate[f"{symbol}_EPS_TTM"]) * 100)).round(2) # 这个是截止下一年的EPS growth rate所得出的PEG ratio, <1是undervalue的表现
    
    df_stock_consolidate[f"{symbol}_nextYear_EPS_growthRate"] = (((EPS_12month_projected - df_stock_consolidate[f"{symbol}_EPS_TTM"]) / df_stock_consolidate[f"{symbol}_EPS_TTM"]) * 100).round(2)
    df_stock_consolidate[f"{symbol}_nextQuater_EPS_growthRate"] = (((df_stock_consolidate[f"{symbol}_EPS_latest_projected"] - df_stock_consolidate[f"{symbol}_EPS_TTM"]) / df_stock_consolidate[f"{symbol}_EPS_TTM"]) * 100).round(2)



    conditions = [
    (df_stock_consolidate[f"{symbol}"] < df_stock_consolidate[f"{symbol}_relative_valuation_TTM_-"]),
    (df_stock_consolidate[f"{symbol}"] > df_stock_consolidate[f"{symbol}_relative_valuation_TTM_+"]),
    ((df_stock_consolidate[f"{symbol}"] >= df_stock_consolidate[f"{symbol}_relative_valuation_TTM_-"]) & (df_stock_consolidate[f"{symbol}"] <= df_stock_consolidate[f"{symbol}_relative_valuation_TTM_+"])),
    ]

    categories = [
        'undervalued'
        ,'overvalued'
        ,'fair'
        ]

    # This KPI assess if the current stock price is under/over/fair to the current relative valuation
    df_stock_consolidate[f"{symbol}_curr_assessment"] = None

    for condition, category in zip(conditions, categories):
        df_stock_consolidate.loc[condition, f"{symbol}_price_valuation_assessment"] = category





    # Append key-value pairs to the dictionary
    selected_cols = [
    f"{symbol}"
    ,f"{symbol}_EPS_TTM"
    ,f"{symbol}_EPS_latest_projected"
    ,f"{symbol}_PE_TTM"
    ,f"{symbol}_PE_TTM_avg"
    ,f"{symbol}_PE_{PE_yr_range-1}yr_avg"
    ,f"{symbol}_PE_{PE_yr_range-1}yr_volatility_+"
    ,f"{symbol}_PE_{PE_yr_range-1}yr_volatility_-"
    ,f"{symbol}_relative_valuation_TTM_+"
    ,f"{symbol}_relative_valuation_TTM_-"
    ,f"{symbol}_relative_valuation_TTM_median"
    ,f"{symbol}_relative_valuation_nextQuater_projected_+"
    ,f"{symbol}_relative_valuation_nextQuater_projected_-"
    ,f"{symbol}_relative_valuation_nextQuater_projected_median"
    ,f"{symbol}_relative_valuation_nextYear_projected_+"
    ,f"{symbol}_relative_valuation_nextYear_projected_-"
    ,f"{symbol}_relative_valuation_nextYear_projected_median"
    ,f"{symbol}_price_valuation_assessment"
    ,f"{symbol}_nextQuater_EPS_growthRate"
    ,f"{symbol}_nextYear_EPS_growthRate"
    ,f"{symbol}_next12months_PEG"
    ,f"{symbol}_TTM_PEG"
    ]

    ticker_dict[f'{symbol}'] = df_stock_consolidate[selected_cols]

    # Addtional setting session
    # Set display options to show all rows and columns
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    warnings.filterwarnings("ignore")
    pd.options.mode.copy_on_write = True

In [8]:
ticker_dict['STZ']

,STZ,STZ_EPS_TTM,STZ_EPS_latest_projected,STZ_PE_TTM,STZ_PE_TTM_avg,STZ_PE_5yr_avg,STZ_PE_5yr_volatility_+,STZ_PE_5yr_volatility_-,STZ_relative_valuation_TTM_+,STZ_relative_valuation_TTM_-,STZ_relative_valuation_TTM_median,STZ_relative_valuation_nextQuater_projected_+,STZ_relative_valuation_nextQuater_projected_-,STZ_relative_valuation_nextQuater_projected_median,STZ_relative_valuation_nextYear_projected_+,STZ_relative_valuation_nextYear_projected_-,STZ_relative_valuation_nextYear_projected_median,STZ_price_valuation_assessment,STZ_nextQuater_EPS_growthRate,STZ_nextYear_EPS_growthRate,STZ_next12months_PEG,STZ_TTM_PEG
2024-07-26,253.48,12.72,13.14,19.93,21.21,22.25,24.47,20.03,282.38,257.20,259.36,291.71,265.69,278.7,337.22,307.14,322.18,undervalued,3.30,19.42,1.64,1.03
2024-07-25,249.38,12.72,13.14,19.61,21.21,22.25,24.47,20.03,282.38,257.20,259.36,291.71,265.69,278.7,337.22,307.14,322.18,undervalued,3.30,19.42,1.64,1.01
2024-07-24,246.88,12.72,13.14,19.41,21.21,22.25,24.47,20.03,282.38,257.20,259.36,291.71,265.69,278.7,337.22,307.14,322.18,undervalued,3.30,19.42,1.64,1.00
2024-07-23,245.98,12.72,13.14,19.34,21.21,22.25,24.47,20.03,282.38,257.20,259.36,291.71,265.69,278.7,337.22,307.14,322.18,undervalued,3.30,19.42,1.64,1.00
2024-07-22,243.21,12.72,13.14,19.12,21.21,22.25,24.47,20.03,282.38,257.20,259.36,291.71,265.69,278.7,337.22,307.14,322.18,undervalued,3.30,19.42,1.64,0.98
2024-07-19,245.45,12.72,13.14,19.30,21.21,22.25,24.47,20.03,282.38,257.20,259.36,291.71,265.69,278.7,337.22,307.14,322.18,undervalued,3.30,19.42,1.64,0.99
2024-07-18,248.26,12.72,13.14,19.52,21.21,22.25,24.47,20.03,282.38,257.20,259.36,291.71,265.69,278.7,337.22,307.14,322.18,undervalued,3.30,19.42,1.64,1.01
2024-07-17,248.23,12.72,13.14,19.51,21.21,22.25,24.47,20.03,282.38,257.20,259.36,291.71,265.69,278.7,337.22,307.14,322.18,undervalued,3.30,19.42,1.64,1.00
2024-07-16,248.09,12.72,13.14,19.50,21.21,22.25,24.47,20.03,282.38,257.20,259.36,291.71,265.69,278.7,337.22,307.14,322.18,undervalued,3.30,19.42,1.64,1.00
2024-07-15,249.53,12.72,13.14,19.62,21.21,22.25,24.47,20.03,282.38,257.20,259.36,291.71,265.69,278.7,337.22,307.14,322.18,undervalued,3.30,19.42,1.64,1.01


In [49]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol=MSFT&apikey={alpha_vantage_api_key}'
r = requests.get(url)
data = r.json()


for key, value in data.items():
    if key == 'annualReports':
        annual_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
        annual_income_df = annual_income_df.sort_values('fiscalDateEnding', ascending=True)

        # annual measurements cols
        annual_income_metric_cols = annual_income_df.columns[1:]

        # Calculate quarter-over-quarter change percentage
        for column in annual_income_metric_cols:  # Exclude the 'fiscalDateEnding' column
            annual_income_df[column] = pd.to_numeric(annual_income_df[column], errors='coerce')
            annual_income_df[f'{column}_YoY'] = annual_income_df[column].pct_change() * 100 
        
        annual_income_YoY_metric_cols = [col for col in annual_income_df.columns if 'YoY' in col]

        # ratios of income statement calculation
        annual_income_df['gross_margin'] = (annual_income_df['grossProfit'] / annual_income_df['totalRevenue']) * 100
        annual_income_df['operating_margin'] = (annual_income_df['operatingIncome'] / annual_income_df['totalRevenue']) * 100
        annual_income_df['net_margin'] = (annual_income_df['netIncome'] / annual_income_df['totalRevenue']) * 100

        annual_income_ratio_cols = [
            'gross_margin'
            ,'operating_margin'
            ,'net_margin'
            ]



    if key == 'quarterlyReports':
        qtr_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
        qtr_income_df = qtr_income_df.sort_values('fiscalDateEnding', ascending=True)

        # qtr measurements cols
        qtr_income_metric_cols = qtr_income_df.columns[1:]
        
        # Calculate quarter-over-quarter change percentage
        for column in qtr_income_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
            qtr_income_df[column] = pd.to_numeric(qtr_income_df[column], errors='coerce')
            qtr_income_df[f'{column}_QoQ'] = qtr_income_df[column].pct_change() * 100

        qtr_income_QoQ_metric_cols = [col for col in qtr_income_df.columns if 'QoQ' in col]

        # ratios of income statement calculation
        qtr_income_df['gross_margin'] = (qtr_income_df['grossProfit'] / qtr_income_df['totalRevenue']) * 100
        qtr_income_df['operating_margin'] = (qtr_income_df['operatingIncome'] / qtr_income_df['totalRevenue']) * 100
        qtr_income_df['net_margin'] = (qtr_income_df['netIncome'] / qtr_income_df['totalRevenue']) * 100

        qtr_income_ratio_cols = [
            'gross_margin'
            ,'operating_margin'
            ,'net_margin'
            ]

In [50]:
annual_income_df

,fiscalDateEnding,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,grossProfit_YoY,totalRevenue_YoY,costOfRevenue_YoY,costofGoodsAndServicesSold_YoY,operatingIncome_YoY,sellingGeneralAndAdministrative_YoY,researchAndDevelopment_YoY,operatingExpenses_YoY,investmentIncomeNet_YoY,netInterestIncome_YoY,interestIncome_YoY,interestExpense_YoY,nonInterestIncome_YoY,otherNonOperatingIncome_YoY,depreciation_YoY,depreciationAndAmortization_YoY,incomeBeforeTax_YoY,incomeTaxExpense_YoY,interestAndDebtExpense_YoY,netIncomeFromContinuingOperations_YoY,comprehensiveIncomeNetOfTax_YoY,ebit_YoY,ebitda_YoY,netIncome_YoY,gross_margin,operating_margin,net_margin
13,2010-06-30,50089000000,62484000000,12395000000,12395000000,24098000000,17218000000,8714000000,26989000000,843000000,-151000000,NaN,151000000,-139000000,14000000,1800000000,707000000,25013000000,6253000000,151000000,18760000000,18846000000,25164000000,25871000000,18760000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.162922,38.566673,30.023686
12,2011-06-30,54366000000,69943000000,15577000000,15577000000,27161000000,18162000000,9043000000,27205000000,900000000,-295000000,NaN,295000000,-103000000,-31000000,2000000000,537000000,28071000000,4921000000,295000000,23150000000,23958000000,28366000000,28903000000,23150000000,8.538801,11.937456,25.671642,25.671642,12.710598,5.482634,3.775534,0.800326,6.761566,95.364238,NaN,95.364238,-25.899281,-321.428571,11.111111,-24.045262,12.225643,-21.301775,95.364238,23.400853,27.125119,12.724527,11.719686,23.400853,77.729008,38.833050,33.098380
11,2012-06-30,56193000000,73723000000,17530000000,17530000000,21763000000,18426000000,9811000000,34430000000,800000000,-380000000,NaN,380000000,-481000000,1000000,2200000000,558000000,22267000000,5289000000,380000000,16978000000,16537000000,22647000000,23205000000,16978000000,3.360556,5.404401,12.537716,12.537716,-19.874084,1.453584,8.492757,26.557618,-11.111111,28.813559,NaN,28.813559,366.990291,-103.225806,10.000000,3.910615,-20.676143,7.478155,28.813559,-26.660907,-30.975040,-20.161461,-19.714217,-26.660907,76.221803,29.519960,23.029448
10,2013-06-30,57464000000,77849000000,20385000000,20385000000,26764000000,20425000000,10411000000,30700000000,677000000,-429000000,NaN,429000000,-270000000,194000000,2600000000,739000000,27052000000,5189000000,429000000,21863000000,22184000000,27481000000,28220000000,21863000000,2.261848,5.596625,16.286366,16.286366,22.979369,10.848801,6.115585,-10.833575,-15.375000,12.894737,NaN,12.894737,-43.866944,19300.000000,18.181818,32.437276,21.489199,-1.890717,12.894737,28.772529,34.147669,21.344991,21.611722,28.772529,73.814693,34.379375,28.083855
9,2014-06-30,59755000000,86833000000,27078000000,16681000000,27759000000,20632000000,11381000000,31996000000,883000000,-597000000,NaN,597000000,-199000000,-169000000,3400000000,845000000,27820000000,5746000000,597000000,22074000000,24039000000,28417000000,29262000000,22074000000,3.986844,11.540290,32.832965,-18.170223,3.717680,1.013464,9.317068,4.221498,30.428360,39.160839,NaN,39.160839,-26.296296,-187.113402,30.769231,14.343708,2.838977,10.734246,39.160839,0.965101,8.361882,3.405990,3.692417,0.965101,68.816003,31.968261,25.421211
8,2015-06-30,60542000000,93580000000,33038000000,21410000000,18161000000,20324000000,12046000000,42381000000,766000000,-781000000,NaN,781000000,-88000000,-267000000,4100000000,1300000000,18507000000,6314000000,781000000,12193000000,11007000000,19288000000,20588000000,12193000000,1.317045,7.770087,22.010488,28.349619,-34.576173,-1.492827,5.843072,32.457182,-13.250283,30.82

In [52]:
qtr_income_df

,fiscalDateEnding,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,grossProfit_QoQ,totalRevenue_QoQ,costOfRevenue_QoQ,costofGoodsAndServicesSold_QoQ,operatingIncome_QoQ,sellingGeneralAndAdministrative_QoQ,researchAndDevelopment_QoQ,operatingExpenses_QoQ,investmentIncomeNet_QoQ,netInterestIncome_QoQ,interestIncome_QoQ,interestExpense_QoQ,nonInterestIncome_QoQ,otherNonOperatingIncome_QoQ,depreciation_QoQ,depreciationAndAmortization_QoQ,incomeBeforeTax_QoQ,incomeTaxExpense_QoQ,interestAndDebtExpense_QoQ,netIncomeFromContinuingOperations_QoQ,comprehensiveIncomeNetOfTax_QoQ,ebit_QoQ,ebitda_QoQ,netIncome_QoQ,gross_margin,operating_margin,net_margin
66,2007-09-30,11087000000,13762000000,2675000000,2675000000,5918000000,718000000,1837000000,5169000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,435000000,6216000000,1927000000,NaN,NaN,NaN,5859000000,6294000000,4289000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.562418,43.002471,31.165528
65,2007-12-31,12824000000,16367000000,3543000000,3543000000,6481000000,1066000000,1885000000,6343000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,481000000,6820000000,2113000000,NaN,NaN,NaN,6348000000,6829000000,4707000000,15.666997,18.928935,32.448598,32.448598,9.513349,48.467967,2.612956,22.712323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.574713,9.716860,9.652309,NaN,NaN,NaN,8.346134,8.500159,9.745862,78.352783,39.597972,28.759088
64,2008-03-31,11940000000,14454000000,2514000000,2514000000,4409000000,2341000000,2035000000,7531000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,532000000,4810000000,422000000,NaN,NaN,NaN,4234000000,4766000000,4388000000,-6.893325,-11.688153,-29.043184,-29.043184,-31.970375,119.606004,7.957560,18.729308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.602911,-29.472141,-80.028396,NaN,NaN,NaN,-33.301827,-30.209401,-6.777140,82.606891,30.503667,30.358378
63,2008-06-30,12971000000,15837000000,2866000000,2866000000,5684000000,887000000,2407000000,7287000000,NaN,NaN,NaN,106000000.0,NaN,NaN,NaN,608000000,5968000000,1671000000,NaN,NaN,NaN,5617000000,6225000000,4297000000,8.634841,9.568286,14.001591,14.001591,28.918122,-62.110209,18.280098,-3.239942,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.285714,24.074844,295.971564,NaN,NaN,NaN,32.664147,30.612673,-2.073838,81.903138,35.890636,27.132664
62,2008-09-30,12213000000,15061000000,2848000000,2848000000,5999000000,887000000,2283000000,6214000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,585000000,5991000000,1618000000,NaN,NaN,NaN,5999000000,6799000000,4373000000,-5.843805,-4.899918,-0.628053,-0.628053,5.541872,0.000000,-5.151641,-14.724852,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,-3.782895,0.385389,-3.171753,NaN,NaN,NaN,6.800783,9.220884,1.768676,81.090233,39.831352,29.035257
61,2008-12-31,12722000000,16629000000,3907000000,3907000000,5939000000,831000000,2290000000,6783000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,632000000,5638000000,1464000000,NaN,NaN,NaN,5939000000,7047000000,4174000000,4.167690,10.410995,37.183989,37.183989,-1.000167,-6.313416,0.306614,9.156743,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,8.034188,-5.892172,-9.517923,NaN,NaN,NaN,-1.000167,3.647595,-4.550652,76.504901,35.714715,25.100728
60,2009-03-31,10834000000,13648000000,2814000000,2814000000,4438000000,1203000000,2212000000,6396000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,664000000,4050000000,1073000000,NaN,NaN,NaN,4438000000,5654000000,2977000000,-14.840434,-17.926514,-27.975429,-27.975429,-25.273615,44.765343,-3.406114,-5.705440,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,5.063291,-28.166016,-26.707650,NaN,NaN,NaN,-25.273615,-19.767277,-28.677528,79.381594,32.517585,21.812720
59,2009-06-30,10513000000,13099000000,2586000000,2586000000,3987000000,1399000000,22

In [33]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Sample data based on the provided image (you should replace this with your actual data)
data = {
    "fiscalDateEnding": ["2024-03-31", "2023-12-31", "2023-09-30", "2023-06-30", "2023-03-31", "2022-12-31", "2022-09-30", "2022-06-30", "2022-03-31"],
    "grossProfit": [43353000000, 42397000000, 40215000000, 39394000000, 36729000000, 35259000000, 34670000000, 35436000000, 33745000000],
    "totalRevenue": [60911000000, 60849000000, 56029000000, 55659000000, 52405000000, 52032000000, 49636000000, 51406000000, 48732000000],
    "costOfRevenue": [28477000000, 29554000000, 23979000000, 26154000000, 24481000000, 27393000000, 22798000000, 25283000000, 23481000000],
    "costofGoodsAndServicesSold": [18505000000, 19623000000, 16302000000, 16795000000, 16128000000, 17488000000, 15452000000, 16429000000, 15615000000],
    "operatingIncome": [27581000000, 27032000000, 26895000000, 24254000000, 22352000000, 20399000000, 21518000000, 20534000000, 20364000000],
    "sellingGeneralAndAdministrative": [8119000000, 8223000000, 6661000000, 8401000000, 7393000000, 8016000000, 6524000000, 8053000000, 7075000000]
}

# Creating a DataFrame
df = pd.DataFrame(data)

# List of columns to plot
columns_to_plot = df.columns[1:]  # Exclude the 'fiscalDateEnding' column
df[f'{}']

# Create bar charts for each column
for column in columns_to_plot:
    fig = px.bar(df, x='fiscalDateEnding', y=column, title=column)
    fig.show()


In [32]:
qtr_income_df

,fiscalDateEnding,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
0,2024-03-31,43353000000,60911000000,28477000000,18505000000,27581000000,8119000000,7653000000,17625000000,619000000,-800000000,619000000,800000000,61711000000,-486000000,None,1853000000,26727000000,4788000000,800000000,21939000000,21453000000,27527000000,29380000000,21939000000
1,2023-12-31,42397000000,60849000000,29554000000,19623000000,27032000000,8223000000,7142000000,17073000000,734000000,-909000000,734000000,909000000,61758000000,-169000000,None,1708000000,26526000000,4656000000,909000000,21870000000,23858000000,27435000000,29143000000,21870000000
2,2023-09-30,40215000000,56029000000,23979000000,16302000000,26895000000,6661000000,6659000000,14336000000,1166000000,-525000000,1166000000,525000000,56554000000,-137000000,None,1016000000,27284000000,4993000000,525000000,22291000000,21697000000,27809000000,28825000000,22291000000
3,2023-06-30,39394000000,55659000000,26154000000,16795000000,24254000000,8401000000,6739000000,16098000000,905000000,-482000000,905000000,482000000,56141000000,-61000000,None,958000000,24727000000,4646000000,482000000,20081000000,19382000000,25209000000,26167000000,20081000000
4,2023-03-31,36729000000,52405000000,24481000000,16128000000,22352000000,7393000000,6984000000,15337000000,748000000,-496000000,748000000,496000000,52901000000,-93000000,None,960000000,22673000000,4374000000,496000000,18299000000,19112000000,23169000000,24129000000,18299000000
5,2022-12-31,35259000000,52032000000,27393000000,17488000000,20399000000,8016000000,6844000000,16749000000,700000000,-490000000,700000000,490000000,52522000000,-38000000,None,1089000000,20339000000,3914000000,490000000,16425000000,17311000000,20829000000,21918000000,16425000000
6,2022-09-30,34670000000,49636000000,22798000000,15452000000,21518000000,6524000000,6628000000,13974000000,641000000,-500000000,641000000,500000000,50136000000,-31000000,None,822000000,21572000000,4016000000,500000000,17556000000,14891000000,22072000000,22894000000,17556000000
7,2022-06-30,35436000000,51406000000,25283000000,16429000000,20534000000,8053000000,6849000000,15703000000,552000000,-496000000,552000000,496000000,51902000000,-23000000,None,801000000,20487000000,3747000000,496000000,16740000000,14538000000,20983000000,21784000000,16740000000
8,2022-03-31,33745000000,48732000000,23481000000,15615000000,20364000000,7075000000,6306000000,14172000000,519000000,-503000000,519000000,503000000,49235000000,-11000000,None,791000000,20190000000,3462000000,503000000,16728000000,13815000000,20693000000,21484000000,16728000000
9,2021-12-31,34768000000,51228000000,24433000000,16960000000,22247000000,6763000000,5758000000,13231000000,503000000,-525000000,503000000,525000000,51753000000,-4000000,None,710000000,22515000000,3750000000,525000000,18765000000,17919000000,23040000000,23750000000,18765000000


# IPO


In [25]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = 'https://www.alphavantage.co/query?function=IPO_CALENDAR&apikey={alpha_vantage_api_key}'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)

    IPO_df = pd.DataFrame(columns=my_list[0]
                          ,data=my_list[1::])

In [5]:
# Function to get the list of S&P 500 companies and their sectors
def get_sp500_companies():
    # Fetch the S&P 500 company symbols and sectors from a reliable source (e.g., Wikipedia)
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    
    # Extract the relevant table containing the company symbols and sectors
    sp500_df = tables[0]
    
    # Return the DataFrame containing S&P 500 companies and sectors
    return sp500_df[['Symbol', 'GICS Sector', 'GICS Sub-Industry']]


# Function to create a dictionary of sectors and sub-sectors
def create_sector_subsector_dict(df):
    sector_subsector_dict = {}
    for index, row in df.iterrows():
        sector = row['GICS Sector']
        subsector = row['GICS Sub-Industry']
        if sector not in sector_subsector_dict:
            sector_subsector_dict[sector] = [subsector]
        else:
            sector_subsector_dict[sector].append(subsector)
    return sector_subsector_dict

# Function to filter the S&P 500 companies by sector
def company_sector_list(df, sector):
    return df[df['GICS Sector'] == sector]['Symbol'].tolist()

def company_sub_sector_list(df, sub_sector):
    return df[df['GICS Sub-Industry'] == sub_sector]['Symbol'].tolist()


# Get the list of S&P 500 companies and their sectors
sp500_df  = get_sp500_companies()

sp500_companies_sectors = sp500_df ['GICS Sector'].value_counts().index
sp500_companies_sub_sectors = sp500_df ['GICS Sub-Industry'].value_counts().index

sector_subsector_dict = create_sector_subsector_dict(sp500_df)


# Function to create a DataFrame from the sector_subsector_dict
def create_sector_dataframe():
    # Create a list to store dictionacompany_sector_listries representing each row of data
    data = []
    
    # Filter the DataFrame to get stocks in the specified sector
    for sector in sp500_companies_sectors:
        sector_stocks_list = company_sector_list(sp500_df, sector)

        # Iterate over the stocks in the sector and create a dictionary for each
        for i, ticker in enumerate(sector_stocks_list, start=1):
            # Create a dictionary for the current stock in the sector
            row_data = {'Sector': sector, 'Ticker': ticker}
            # Append the dictionary to the list
            data.append(row_data)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df


sector_subsector_dict = create_sector_subsector_dict(sp500_df)


sector_ticker_df = create_sector_dataframe()
sector_ticker_list = sector_ticker_df[sector_ticker_df['Sector'] == 'Consumer Staples']['Ticker'].tolist()

In [6]:
# Pivot the DataFrame
pivot_sector_ticker_df = sector_ticker_df.groupby('Sector')['Ticker'].apply(list).reset_index()

# Transpose to get sectors as columns
pivot_sector_ticker_df = pivot_sector_ticker_df.set_index('Sector').T

pivot_sector_ticker_df

Sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Ticker,"[GOOGL, GOOG, T, CHTR, CMCSA, EA, FOXA, FOX, I...","[ABNB, AMZN, APTV, AZO, BBWI, BBY, BKNG, BWA, ...","[MO, ADM, BF.B, BG, CPB, CHD, CLX, KO, CL, CAG...","[APA, BKR, CVX, COP, CTRA, DVN, FANG, EOG, EQT...","[AFL, ALL, AXP, AIG, AMP, AON, ACGL, AJG, AIZ,...","[ABT, ABBV, A, ALGN, AMGN, BAX, BDX, BIO, TECH...","[MMM, AOS, ALLE, AAL, AME, ADP, AXON, BA, BR, ...","[ACN, ADBE, AMD, AKAM, APH, ADI, ANSS, AAPL, A...","[APD, ALB, AMCR, AVY, BALL, CE, CF, CTVA, DOW,...","[ARE, AMT, AVB, BXP, CPT, CBRE, CSGP, CCI, DLR...","[AES, LNT, AEE, AEP, AWK, ATO, CNP, CMS, ED, C..."


In [15]:
pivot_sector_ticker_df['Financials'].iloc[0]

['AFL',
 'ALL',
 'AXP',
 'AIG',
 'AMP',
 'AON',
 'ACGL',
 'AJG',
 'AIZ',
 'BAC',
 'BK',
 'BRK.B',
 'BLK',
 'BX',
 'BRO',
 'COF',
 'CBOE',
 'SCHW',
 'CB',
 'CINF',
 'C',
 'CFG',
 'CME',
 'CPAY',
 'DFS',
 'EG',
 'FDS',
 'FIS',
 'FITB',
 'FI',
 'BEN',
 'GPN',
 'GL',
 'GS',
 'HIG',
 'HBAN',
 'ICE',
 'IVZ',
 'JKHY',
 'JPM',
 'KEY',
 'KKR',
 'L',
 'MTB',
 'MKTX',
 'MMC',
 'MA',
 'MET',
 'MCO',
 'MS',
 'MSCI',
 'NDAQ',
 'NTRS',
 'PYPL',
 'PNC',
 'PFG',
 'PGR',
 'PRU',
 'RJF',
 'RF',
 'SPGI',
 'STT',
 'SYF',
 'TROW',
 'TRV',
 'TFC',
 'USB',
 'V',
 'WRB',
 'WFC',
 'WTW']

In [16]:
len(pivot_sector_ticker_df['Financials'].iloc[0])

71

In [32]:
ticker_symbols

['TSM', 'ACN']

## Appendix

In [128]:
# Consumer Staples

ticker_symbols = [
    'MO',
    'ADM',
    # 'BF.B', # DATA ISSUE
    'BG',
    'CPB',
    'CHD',
    'CLX',
    'KO',
    'CL',
    'CAG',
    'STZ', # !!
    'COST',
    'DG',
    'DLTR', # !!
    'EL',
    'GIS',
    'HSY',
    'HRL',
    'K',
    'KVUE',
    'KDP',
    'KMB',
    'KHC',
    # 'KR', # DATA ISSUE
    'LW',
    # 'MKC', # DATA ISSUE
    'TAP',
    'MDLZ',
    'MNST',
    'PEP',
    'PM',
    'PG',
    'SJM',
    'SYY',
    'TGT', # !!
    'TSN', # !!
    # 'WBA', # DATA ISSUE
    'WMT'
]

In [ ]:
# Energy

ticker_symbols = [
'APA',
 'BKR', # !! HIGH PE, LOW PROFITABILITY, HIGH FORECAST,  BUT CANNOT BEAT FORECAST, 总结与TRGP类似, 基本面更稳妥, 盈利预期高, 有待考证
 'CVX',
 'COP',
 'CTRA',
 'DVN',
 'FANG',
 'EOG',
 'EQT',
 'XOM',
 'HAL', # !!!! 与SLB类似
 'HES', # !  HIGH PE, UNDERVALUE, 市场炒作高预期, 2024表现强劲, 已经被超热了
 'KMI',
 'MRO',
 'MPC',
 'OXY',
 'OKE', # !!
 'PSX',
 'SLB', # !!! A BIT HIGH PE, GOOD ROE, MED PROFITABILITY, STEADY GROWTH FORECAST, AND CAN BEAT FORECAST, 总结基本面良好，与HAL类型相似, 市场炒作热度比HAL更高, 值得研究
 'TRGP', # !！ HGIH PE, HIGH ROE, RELATIVE MORE DEBT THAN EQUITY, LOW PROFITABILITY, HIGH FORECAST, BUT CANNOT BEAT FORECAST, 总结基本面一般, 但是盈利预期非常高, 有待考证
 'VLO',
 'WMB'
 ]

In [21]:
symbol = 'STZ'

# for i in earning_calendar: comment out the for loop in case of future usage, i can be the parameter of {}month
CSV_URL = f'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={symbol}&horizon=12month&apikey={alpha_vantage_api_key}'
with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)

In [22]:
my_list[1]

['STZ',
 'Constellation Brands Inc - Class A',
 '2024-10-03',
 '2024-08-31',
 '4.12',
 'USD']